# Setup

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

fb = pd.read_csv(
    'data/fb_stock_prices_2018.csv', 
    index_col='date', 
    parse_dates=True
)

quakes = pd.read_csv('data/earthquakes.csv')

covid = pd\
    .read_csv('data/covid19_cases.csv')\
    .assign(
        date=lambda x: pd.to_datetime(x.dateRep, format='%d/%m/%Y')
    )\
    .set_index('date')\
    .replace('United_States_of_America', 'USA')\
    .sort_index()['2020-01-18':'2020-09-18']

# Exercise 1

Using seaborn , create a heatmap to visualize the correlation coefficients between
earthquake magnitude and whether there was a tsunami for earthquakes measured
with the mb magnitude type.

# Exercise 2

Create a box plot of Facebook volume traded and closing prices, and draw reference
lines for the bounds of a Tukey fence with a multiplier of 1.5. The bounds will be
at Q 1 − 1.5 × IQR and Q 3 + 1.5 × IQR. Be sure to use the quantile() method on
the data to make this easier. (Pick whichever orientation you prefer for the plot, but
make sure to use subplots.)

# Exercise 3

Plot the evolution of cumulative COVID-19 cases worldwide, and add a dashed
vertical line on the date that it surpassed 1 million. Be sure to format the tick labels
on the y-axis accordingly

# Exercise 4

Use axvspan() to shade a rectangle from '2018-07-25' to '2018-07-31' ,
which marks the large decline in Facebook price on a line plot of the closing price.

# Exercise 5



**Using the Facebook stock price data, annotate the following three events on a line
plot of the closing price**

a) Disappointing user growth announced after close on July 25, 2018

b) Cambridge Analytica story breaks on March 19, 2018 (when it affected the market)

c) FTC launches investigation on March 20, 2018

# Exercise 6



Modify the reg_resid_plots() function to use a matplotlib colormap
instead of cycling between two colors. Remember, for this use case, we should
pick a qualitative colormap or make our own.

<hr>
<div style="overflow: hidden; margin-bottom: 10px;">
    <div style="float: left;">
        <a href="./python_101.ipynb">
            <button>Python 101</button>
        </a>
    </div>
    <div style="float: right;">
        <a href="../../solutions/ch_01/solutions.ipynb">
            <button>Solutions</button>
        </a>
        <a href="../ch_02/1-pandas_data_structures.ipynb">
            <button>Chapter 2 &#8594;</button>
        </a>
    </div>
</div>
<hr>